# Strain mapping of simulated Ge/SiGe multilayer stacks

This notebook measures the strain fields of a simulated 4D-STEM dataset  which consists of alternating $Si / Si_{50}Ge_{50}$ multilayer stacks, on a $Si_{25}Ge_{75}$ substrate using both FCU-Net and a correlation based approach implemented in py4DSTEM library.  This notebook uses the "ideal" simulation, where the sample is aligned perfectly along the zone axis. All of the non-uniformity of the diffracted disks is due to multiple scattering.

[Download the simulated Ge/SiGe multilayer dataset with an ideal structure, after it has been cropped.](https://drive.google.com/file/d/1C6rPB9KpNML_w1wbZrhUE0MORhxDD6_V/view?usp=sharing)

### Simulation Parameters

|Parameter | Value | Units |
|:--- |:--- |:--- |
| accelerating voltage | 200 | kV |
| wavelength | 0.02508 | Ang |
| convergence semiangle | 2 | mrads |
| cell dimensions | (1008.28,   253.58,  404.14) | Ang |
| algorithm | PRISM |  |
| interpolation factors | (12, 3) |  |
| scan range x | (0.1, 0.9) | Ang |
| scan range y | (0.1, 0.9) | Ang |
| probe step x | 4.03 | Ang |
| probe step y | 4.06 | Ang |

### Acknowledgements

This tutorial was written by  Alex Rakowski (arakowski@lbl.gov) with assistance from Ben Savitzky (bhsavitzky@lbl.gov), Joydeep Munshi, and Colin Ophus (clophus@lbl.gov) at Berkeley Lab.

The dataset was simulated by Alexander Rakowski (arakowski@lbl.gov) and Colin Ophus (clophus@lbl.gov), using the [Prismatic simulation code](https://prism-em.com/), with help from Luis Rangel DaCosta (luisrd@berkeley.edu ).

### Version
Last updated with py4DSTEM version 0.13.14

## Download Data

In [ ]:
# #Download the required database directly from google drive
from py4DSTEM.io import download_file_from_google_drive
# #Change the 2nd argument of the function according to location of your local drive
download_file_from_google_drive("1tqYFJN1GHatOu8blu4s9X_TdnWIgkr0F", "./Si_SiGe_ideal_50x200_201x50x256x256.h5")
download_file_from_google_drive("1-tLGhJyCqMxxz5gg9_R6MfowOK5KI_hv", "./strain_profile_simulated.csv")

## Imports 

In [ ]:
# Setting a couple of environment parameters to handle the Tensorflow Behaviour
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true" 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
import tensorflow_addons as tfa
import py4DSTEM
import crystal4D
import matplotlib.pyplot as plt
import numpy as np
import h5py
import cupy as cp

## Check the install appears correct:

In [ ]:
print(f"py4DSTEM version: {py4DSTEM.__version__}")
print(f"Crystal4D version: {crystal4D.__version__}")
print(f"Tensorflow version: {tf.__version__}")
print(f"Tensorflow-Addons version: {tfa.__version__}")
print(f"Cupy version: {cp.__version__}")
print(f"Number of GPUs detected: {len(tf.config.list_physical_devices('GPU'))}")
print("Cuda envionment:", "cudatoolkit: 11.0.3", "cudnn: 8.1.0.77", sep='\n') 

## Set file Paths

In [ ]:
# File paths - output files will be augmented with the correct file extension
file_path_input = './Si_SiGe_ideal_50x200_201x50x256x256.h5'
file_path_output = './Si_SiGe_ideal_analysis.h5'

## Load the Files

In [ ]:
with h5py.File(file_path_input, 'r') as f:
    # print(f.keys())
    probe = py4DSTEM.io.Probe(data = f['probe'][...])
    datacube = py4DSTEM.io.DataCube(data = np.moveaxis(f['datacube'][...], (0,1), (1,0)))
#     datacube = py4DSTEM.io.DataCube(data = f['datacube'][...])

## Plot Median Diffraction Pattern

In [ ]:
# calculate the mean CBED
datacube.get_dp_mean(returncalc=False) 
# plot the mean CBED
py4DSTEM.show(
    datacube.tree['dp_mean'],
    scaling='power',
    power=0.5,
    cmap='inferno')

In [ ]:
# Estimate the radius of the BF disk, and the center coordinates
probe_semiangle, probe_qx0, probe_qy0 = probe.get_probe_size(thresh_upper=0.9, )
print(f'Estimated probe radius = {probe_semiangle.round(3)}')

## Generate Virtual Images

In [ ]:
# Generate some Virtual Images 
expand_BF = 4.0

center = (probe_qx0, probe_qy0)
radius = probe_semiangle + expand_BF


datacube.get_virtual_image(
    mode = 'circle',
    geometry = (center,radius),
    name = 'bright_field',
)


radii = (probe_semiangle + expand_BF, 1e3)

datacube.get_virtual_image(
    mode = 'annulus',
    geometry = (center,radii),
    name = 'dark_field',
);

In [ ]:
py4DSTEM.show(
    datacube.tree['bright_field'].data,
    figsize=(16,2),
    bordercolor = 'w',
    cmap='gray',
    title='Virtual Bright Field',
)
py4DSTEM.show(
    datacube.tree['dark_field'].data,
    figsize=(16,2),
    bordercolor = 'w',
    cmap = 'gray',
    title='Virtual Dark Field',
)

## Get the Probe Kernel

In [ ]:
probe.get_kernel(
    mode='sigmoid', 
    radii = (probe_semiangle*1.0, probe_semiangle * 4.0),
    returncalc=False)

py4DSTEM.visualize.show_kernel(
    probe.kernel,
    R = 20,
    L = 20,
    W = 1
)

## Disk Detection

In [ ]:
# Choose a subset of diffraction patterns to use for hyperparameter tuning

rxs = 25, 25, 25, 25, 25, 25
rys = 17, 39, 74, 94, 120, 160

# rxs = 17, 39, 74, 94, 163, 180
# rys = 25,25,25,25,25,25
colors=['r','limegreen','c','g','orange', 'violet']

py4DSTEM.visualize.show_points(
    datacube.tree['dark_field'],
    x=rxs,
    y=rys,
    pointcolor=colors,
    figsize=(8,8)
)

In [ ]:
# Test hyperparameters on a few probe positions
# Visualize the diffraction patterns and the located disk positions


# Hyperparameters
detect_params_corr = {
    'corrPower': 1.0,
    'sigma': 0,
    'edgeBoundary': 16,
    'minRelativeIntensity': 0.001,
    'minAbsoluteIntensity':2e-6,
    'minPeakSpacing': 20,
    'subpixel' : 'multicorr',
    'upsample_factor': 32,
    'maxNumPeaks': 200,
}


pixelSizeInvAng = 0.0217
detect_params_ml = {
    'minPeakSpacing': 0.45/pixelSizeInvAng, 
    'minRelativeIntensity': 0, 
    'minAbsoluteIntensity': 16, 
    'edgeBoundary': 4,
    'subpixel': 'multicorr',
    'upsample_factor': 32,
}

disks_selected = datacube.find_Bragg_disks(
    data = (rxs, rys),
    template = probe.kernel,
    **detect_params_corr,
)


py4DSTEM.visualize.show_image_grid(
    get_ar = lambda i:datacube.data[rxs[i],rys[i],:,:],
    H=2, 
    W=3,
    axsize=(5,5),
    intensity_range='absolute',
    vmin=0,
    vmax=5e-5,
    # scaling='power',
    # power=0.5,
    get_bordercolor = lambda i:colors[i],
    get_x = lambda i: disks_selected[i].data['qx'],
    get_y = lambda i: disks_selected[i].data['qy'],
    get_pointcolors = lambda i: colors[i],
    open_circles = True,
    scale = 700,
)

## Using *traditional* Cross correlation method

In [ ]:
datacube.find_Bragg_disks(template=probe.kernel, 
                          name='bv_cor', 
                          **detect_params_corr,
                          returncalc=False)

In [ ]:
# we can see that the results are automagically added to the datacube
datacube.tree

## Using FCU-Net Machine Learning method

In [ ]:
datacube.find_Bragg_disks(template=probe.probe, 
                          name='bv_ml',
                          ML=True, 
                          CUDA = True,
                          **detect_params_ml,
                          returncalc=False)

In [ ]:
# Again the results are added automagically to the datacube
datacube.tree

# Working with Bragg Vectors

In [ ]:
# create separate varabiles 
bragg_vectors_corr = datacube.tree['bv_cor']
bragg_vectors_ml = datacube.tree['bv_ml']

# Center coordinate system


In [ ]:
# Compute a Bragg vector map (BVM), 
# a 2D histogram of the Bragg peak positions, 
# weighted by their correlation intensities
bragg_vectors_ml.get_bvm(mode='raw')
bragg_vectors_corr.get_bvm(mode='raw');

In [ ]:
# Compute a Bragg vector map (BVM), 
# a 2D histogram of the Bragg peak positions, 
# weighted by their correlation intensities

# Plot BVMs
bvm_vis_params_ml = {
#     'scaling':'power',
#     'power':0.5,
    'intensity_range':'absolute',
    'vmin':0,
    'vmax':50,
    'cmap':'inferno',
    'figsize': (4,4),
}
py4DSTEM.show(
    bragg_vectors_ml.bvm_raw,
    **bvm_vis_params_ml
)

bvm_vis_params_corr = {
#     'scaling':'power',
#     'power':0.5,
    'intensity_range':'absolute',
    'vmin':0,
    'vmax':1e-2, # note how much lower the intensity is than ML BVM
    'cmap':'inferno',
    'figsize': (4,4),
}

py4DSTEM.show(
    bragg_vectors_corr.bvm_raw,
    **bvm_vis_params_corr
)

In [ ]:
center_guess = (probe_qx0, probe_qy0)
# Compute the origin position pattern-by-pattern
origin_meas_ml = bragg_vectors_ml.measure_origin(
    mode = 'no_beamstop',
    center_guess = center_guess,
)


origin_meas_corr = bragg_vectors_corr.measure_origin(
    mode = 'no_beamstop',
    center_guess = center_guess,
)

# Some local variation in the position of the origin due to electron-sample interaction is
# expected, and constitutes meaningful signal that we would not want to subtract away.
# In fitting a plane or parabolic surface to the measured origin shifts, we aim to
# capture the systematic shift of the beam due to the changing scan coils,
# while removing as little physically meaningful signal we can.

qx0_fit_ml,qy0_fit_ml,qx0_residuals_ml,qy0_residuals_ml = bragg_vectors_ml.fit_origin()

qx0_fit_corr,qy0_fit_corr,qx0_residuals_corr,qy0_residuals_corr = bragg_vectors_corr.fit_origin()

In [ ]:
# Center the disk positions about the origin
bragg_vectors_ml.calibrate()
bragg_vectors_corr.calibrate()

In [ ]:
# see how the x and y coordinates are now centered 
print(bragg_vectors_corr.vectors_uncal.get_pointlist(0,0).data[0])
print(bragg_vectors_corr.vectors.get_pointlist(0,0).data[0])

In [ ]:
bragg_vectors_ml.get_bvm(mode='centered')
bragg_vectors_corr.get_bvm(mode='centered')

# Plot the BVM
bvm_vis_params_ml = {
#     'scaling':'power',
#     'power':0.5,
    'intensity_range':'absolute',
    'vmin':0,
    'vmax':50,
    'cmap':'inferno',
#     'figsize': (4,4),
}
py4DSTEM.show(
    bragg_vectors_ml.bvm_centered,
    **bvm_vis_params_ml
)

bvm_vis_params_corr = {
#     'scaling':'power',
#     'power':0.5,
    'intensity_range':'absolute',
    'vmin':0,
    'vmax':1e-2,
    'cmap':'inferno',
#     'figsize': (4,4),
}

py4DSTEM.show(
    bragg_vectors_corr.bvm_centered,
    **bvm_vis_params_corr
)

In [ ]:
bragg_vectors_corr.choose_lattice_vectors(
    0,
    3,
    2,
    sigma=0, 
    minSpacing=13,
    minAbsoluteIntensity=1e-4,
    maxNumPeaks=100,
    subpixel='multicorr',
    bvm_vis_params = bvm_vis_params_corr,
)


bragg_vectors_ml.choose_lattice_vectors(
    0,
    5,
    3,
    sigma=0, 
    minSpacing=13,
    minAbsoluteIntensity=1e-1,
    maxNumPeaks=100,
    subpixel='multicorr',
    bvm_vis_params = bvm_vis_params_ml,
)

In [ ]:
bragg_vectors_corr.index_bragg_directions(bvm_vis_params = bvm_vis_params_corr)
bragg_vectors_ml.index_bragg_directions(bvm_vis_params = bvm_vis_params_ml)

In [ ]:
# maximum peak spacing from expected positions
max_peak_spacing = 2

# add the lattice indices to all Bragg peaks
bragg_vectors_corr.add_indices_to_braggpeaks(
    maxPeakSpacing = max_peak_spacing,
)

bragg_vectors_ml.add_indices_to_braggpeaks(
    maxPeakSpacing = max_peak_spacing,
)

In [ ]:
# loop through all probe positions and find the best fit lattice

bragg_vectors_corr.fit_lattice_vectors_all_DPs()
bragg_vectors_ml.fit_lattice_vectors_all_DPs()

In [ ]:
# To calculate strain, we need to find how lattice vector changes from a reference region.

# Initially, we will define the reference region to be all pixels, and compute the strain
mask_ml = np.ones((bragg_vectors_ml.shape[0],bragg_vectors_ml.shape[1]),dtype=bool)
bragg_vectors_ml.get_strain_from_reference_region(mask = mask_ml )

# Calculate the strain maps, referenced to the median lattice vectors of all probe positions
strainmap_median_ml = bragg_vectors_ml.get_rotated_strain_map(mode = 'median')

# plot the 4 components of the strain tensor
py4DSTEM.visualize.show_strain(
    strainmap_median_ml,
    vrange_exx = [-3.0, 3.0],
    vrange_theta = [-3.0, 3.0],
    ticknumber = 3,
    axes_plots = (),
    bkgrd = False,
    figsize = (14,4)
)


# To calculate strain, we need to find how lattice vector changes from a reference region.

# Initially, we will define the reference region to be all pixels, and compute the strain
mask_corr = np.ones((bragg_vectors_corr.shape[0],bragg_vectors_corr.shape[1]),dtype=bool)
bragg_vectors_corr.get_strain_from_reference_region(mask = mask_corr )

# Calculate the strain maps, referenced to the median lattice vectors of all probe positions
strainmap_median_corr = bragg_vectors_corr.get_rotated_strain_map(mode = 'median')

# plot the 4 components of the strain tensor
py4DSTEM.visualize.show_strain(
    strainmap_median_corr,
    vrange_exx = [-3.0, 3.0],
    vrange_theta = [-3.0, 3.0],
    ticknumber = 3,
    axes_plots = (),
    bkgrd = False,
    figsize = (14,4)
)



In [ ]:
# Choose a reference lattice location to be the probe positions inside the substrate.
x0,xf = 0,50
y0,yf = 150, 200

py4DSTEM.show(
    strainmap_median_ml.get_slice('e_yy').data,              
    mask = mask_ml,
    figsize = (7, 20), 
    cmap = 'RdBu',
    intensity_range = 'absolute',
    vmin = -0.1,
    vmax = 0.1,
    rectangle={'lims':(x0,xf,y0,yf),'fill':False,'color':'k'}
)

py4DSTEM.show(
    strainmap_median_corr.get_slice('e_yy').data,              
    mask = mask_corr,
    figsize = (7, 20), 
    cmap = 'RdBu',
    intensity_range = 'absolute',
    vmin = -0.1,
    vmax = 0.1,
    rectangle={'lims':(x0,xf,y0,yf),'fill':False,'color':'k'}
)

In [ ]:
# Get new reference lattice vectors
mask_ml[:] = False
mask_ml[x0:xf,y0:yf] = True

mask_corr[:] = False
mask_corr[x0:xf,y0:yf] = True

bragg_vectors_ml.get_strain_from_reference_g1g2(mask_ml)
bragg_vectors_corr.get_strain_from_reference_g1g2(mask_corr)


strainmap_ROI_ml = bragg_vectors_ml.get_rotated_strain_map(mode = 'reference')
strainmap_ROI_corr = bragg_vectors_corr.get_rotated_strain_map(mode = 'reference')

# plot the 4 components of the strain tensor
fig,axs = py4DSTEM.visualize.show_strain(
    strainmap_ROI_ml,
    vrange_exx = [-3.0, 3.0],
    vrange_theta = [-3.0, 3.0],
    ticknumber = 3,
    axes_plots = (),
    bkgrd = False,
    figsize = (14,5),
    returnfig = True
)


# plot the 4 components of the strain tensor
fig,axs = py4DSTEM.visualize.show_strain(
    strainmap_ROI_corr,
    vrange_exx = [-3.0, 3.0],
    vrange_theta = [-3.0, 3.0],
    ticknumber = 3,
    axes_plots = (),
    bkgrd = False,
    figsize = (14,5),
    returnfig = True
)

# Plot line traces for the mean strain values

In [ ]:
# get the ideal strain profile from csv file
import pandas as pd
strain_prof = pd.read_csv('./strain_profile_simulated.csv')
probe_step_x = strain_prof['position']
strain_ideal_e_xx =  strain_prof['strain']
strain_ideal_e_yy = np.zeros_like(probe_step_x)

In [ ]:
# Note to save time, we will just manually specify the probe positions
probe_step_x_ = np.arange(0,strainmap_ROI_ml.data.shape[1]) * 4.033104 + 100.82759

In [ ]:
strain_ideal_e_xx.shape,strain_mean_e_xx_ml.shape

In [ ]:
# Get mean E_xx and E_yy strain maps along the perpendicular to growth direction
strain_mean_e_xx_ml = np.mean(strainmap_ROI_ml['e_xx'].data,axis=0)
strain_mean_e_yy_ml = np.mean(strainmap_ROI_ml['e_yy'].data,axis=0)

strain_mean_e_xx_corr= np.mean(strainmap_ROI_corr['e_xx'].data,axis=0)
strain_mean_e_yy_corr = np.mean(strainmap_ROI_corr['e_yy'].data,axis=0)



# Plotting
fig, axs = plt.subplots(1,2,figsize=(16,4))

# strain along x direction
axs[0].plot(
    probe_step_x, 
    strain_ideal_e_xx,
    color='k',
    linestyle=':');
axs[0].plot(
    probe_step_x_, 
    strain_mean_e_xx_ml+1,
    color='r',
    label='ML')
axs[0].plot(
    probe_step_x_, 
    strain_mean_e_xx_corr+1,
    color='g',
    label='Corr')
axs[0].set_xlim(95,900)
axs[0].set_ylim(0.975,1.015)
axs[0].legend()
# strain along y direction
axs[1].plot(
    probe_step_x, 
    strain_ideal_e_yy,
    color='k',
    linestyle=':');
axs[1].plot(
    probe_step_x_, 
    strain_mean_e_yy_ml,
    color='r',
    label='ML')
axs[1].plot(
    probe_step_x_, 
    strain_mean_e_yy_corr,
    color='g',
    label='Corr')
axs[1].set_xlim(95,900)
axs[1].set_ylim(-0.03, 0.03)
plt.show()

# Bonus: Accessing the FCU-Net model as a tensorflow model

In [ ]:
from py4DSTEM.process.diskdetection.diskdetection_aiml import _get_latest_model
# from py4DSTEM.process.diskdetection.diskdetection import find_Bragg_disks_aiml, get_maxima_2D

# Load the Model
model = _get_latest_model()

# Looking at the inputs to the Model
plt.imshow(
    np.hstack([probe.probe*10, 
               datacube.data[12,0]**0.5,
               ]))
plt.show()

# Predict with the model shape of inputs must be Bx256x256x2,
# where B is the Batchsize
ml_out = model.predict([probe.probe.reshape(1,256,256,1), datacube.data[12,0].reshape(1,256,256,1)])

plt.imshow(ml_out[0,...,0],
          vmin=0,
          vmax=1e2)
plt.show()